In [1]:
import os
import sys
from win32com.client.gencache import EnsureDispatch, EnsureModule
from win32com.client import CastTo, constants

http://www.zemax.com/support/resource-center/knowledgebase/how-to-build-and-optimize-a-singlet-using-zos-api

In [2]:
EnsureModule('ZOSAPI_Interfaces', 0, 1, 0)
connect = EnsureDispatch('ZOSAPI.ZOSAPI_Connection')

# TheApplication
app = connect.CreateNewApplication()  # TheApplication

# TheSystem
osys = app.PrimarySystem  
osys.New(False)
samplesDir = app.SamplesDir
file_out = os.path.join(samplesDir, 'Sequential', 'Ojbectives', 
                       'Single Lens Example wizard+EFFL.zmx')
osys.SaveAs(file_out)

# Aperture
osys_data = osys.SystemData 
osys_data.Aperture.ApertureValue = 40

# Fields
osys_data.Fields.AddField(0, 5.0, 1.0);

# Setting wavelength using wavelength preset
osys_data.Wavelengths.SelectWavelengthPreset(constants.WavelengthPreset_d_0p587);

# Lens data
lde = osys.LDE
lde.InsertNewSurfaceAt(1)
lde.InsertNewSurfaceAt(1)
surf1 = lde.GetSurfaceAt(1)
surf2 = lde.GetSurfaceAt(2)
surf3 = lde.GetSurfaceAt(3)
surf1.Thickness = 10.0
surf1.Comment = 'front of lens'
surf1.Material = 'N-BK7'
surf2.Thickness = 50.0
surf2.Comment = 'rear of lens'
surf3.Thickness = 350.0
surf3.Comment = 'Stop is free to move'

# Setting solves - Made thickness and radii variable
surf1.ThicknessCell.MakeSolveVariable()
surf2.RadiusCell.MakeSolveVariable()
surf2.ThicknessCell.MakeSolveVariable()
surf3.ThicknessCell.MakeSolveVariable()

# Setting up the default merit function 
mfe = osys.MFE
wizard = mfe.SEQOptimizationWizard
wizard.Type = 0          # RMS
wizard.Data = 1          # Spot Radius
wizard.Reference = 0     # Centroid
wizard.Ring = 2          # 3 Rings
wizard.Arm = 0           # 6 Arms
wizard.IsGlassUsed = True
wizard.GlassMin = 3
wizard.GlassMax = 15
wizard.GlassEdge = 3
wizard.IsAirUsed = True
wizard.AirMin = 0.5
wizard.AirMax = 1000
wizard.AirEdge = 0.5
wizard.IsAssumeAxialSymmetryUsed = True
wizard.CommonSettings.OK()

operand1 = mfe.InsertNewOperandAt(1)
operand1.ChangeType(constants.MeritOperandType_EFFL)
operand1.Target = 400.0
operand1.Weight = 1.0

# Local optimization
local_opt = osys.Tools.OpenLocalOptimization()
local_opt.Algorithm = constants.OptimizationAlgorithm_DampedLeastSquares
local_opt.Cycles = constants.OptimizationCycles_Automatic
local_opt.NumberOfCores = 8
base_tool = CastTo(local_opt, 'ISystemTool')
base_tool.RunAndWaitForCompletion()
base_tool.Close()

# save
osys.Save()
app.CloseApplication()
del app

In [24]:
type(local_opt)

win32com.gen_py.EA433010-2BAC-43C4-857C-7AEAC4A8CCE0x0x1x0.ILocalOptimization.ILocalOptimization

In [27]:
# Methods of local_opt
[each for each in dir(local_opt) if not each.startswith('_')]

['CLSID', 'coclass_clsid']

In [28]:
# Also, if we try to access these methods .....
local_opt.RunAndWaitForCompletion()

AttributeError: '<win32com.gen_py.ZOSAPI_Interfaces.ILocalOptimization instance at 0x108227888>' object has no attribute 'RunAndWaitForCompletion'

From above it sees that `ILocalOptimization` interface doesn't have any methods! Actually, this is because of the way the Python COM interface handles inherience. As it turns out, `ILocalOptimization` is inherited from `ISystemTool` interface (so ideally, we should have be able to access all the inherited methods like run(), close(), etc., directly, but it seems that the Python COM interface cannot do it transparently). Therefore, we need to cast the derived calss object `ILocalOptimization` to its base class object `ISystemTool` to get access to those methods. 

True

True

Analyses

In [40]:
ana = osys.Analyses

In [41]:
fft_mtf_ana = ana.New_FftMtf()

In [42]:
type(fft_mtf_ana)

win32com.gen_py.EA433010-2BAC-43C4-857C-7AEAC4A8CCE0x0x1x0.IA_.IA_

The object returned by `ana.New_FftMtf()` is `IA_` the base interface for all analysis window.

In [43]:
# Methods of fft_mtf_ana
[each for each in dir(fft_mtf_ana) if not each.startswith('_')]

['Apply',
 'ApplyAndWaitForCompletion',
 'CLSID',
 'Close',
 'GetResults',
 'GetSettings',
 'IsRunning',
 'Release_2',
 'Terminate',
 'ToFile',
 'WaitForCompletion',
 'coclass_clsid']

In [46]:
# properties of fft_mtf_ana
list(fft_mtf_ana._prop_map_get_.keys())

['StatusMessages',
 'Title',
 'HasAnalysisSpecificSettings',
 'GetAnalysisName',
 'AnalysisType']

In [47]:
# Get the settings for the current analysis
ana_settings = fft_mtf_ana.GetSettings()

In [48]:
type(ana_settings)

win32com.gen_py.EA433010-2BAC-43C4-857C-7AEAC4A8CCE0x0x1x0.IAS_.IAS_

The method `fft_mtf_ana.GetSettings()` returns `IAS_` Interface which is the base class for all analysis settings interfaces 

In [49]:
# Methods of ana_settings
[each for each in dir(ana_settings) if not each.startswith('_')]

['CLSID',
 'Load',
 'LoadFrom',
 'ModifySettings',
 'Reset',
 'Save',
 'SaveTo',
 'Verify',
 'coclass_clsid']

In [50]:
# properties of ana_settings
list(ana_settings._prop_map_get_.keys())

[]

In [6]:
# Handling constants

#app.LicenseStatus
#constants.LicenseStatusType_PremiumEdition
# Setting wavelength with wavelength preset
constants.WavelengthPreset_d_0p587

5

However there is actually a `IAS_FftMtf` Interface that 

In [51]:
ana_mtf_settings = CastTo(ana_settings, 'IAS_FftMtf')

In [52]:
ana_mtf_settings.MaximumFrequency

0.0

In [53]:
ana_mtf_settings.MaximumFrequency = 50

In [ ]:
EnsureModule('ZOSAPI_Interfaces', 0, 1, 0)
connect = EnsureDispatch('ZOSAPI.ZOSAPI_Connection')

# TheApplication
app = connect.CreateNewApplication()  # TheApplication

# TheSystem
osys = app.PrimarySystem  